# Ollama PDF RAG Notebook

## Import & Install Libraries


In [1]:
!pip install langchain_community langchain_ollama langchain_text_splitters langchain_core
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm
%pip install pymupdf
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr poppler-utils
!sudo apt-get install -y tesseract-ocr-hin tesseract-ocr-urd tesseract-ocr-ben tesseract-ocr-eng tesseract-ocr-mar tesseract-ocr-chi-sim
!pip install pytesseract pdf2image Pillow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app


In [2]:
# Imports
from langchain_community.document_loaders import PyMuPDFLoader, OnlinePDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

import pytesseract
from pdf2image import convert_from_path
import os


from langchain_core.documents import Document

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Combined Code for scanned and non-scanned pdfs
##### *Remember to specify the language_code and path to pdf

In [24]:
def extract_text_from_pdf(pdf_path, language='eng', min_word_threshold=100):
    """
    Extracts text from a PDF file using PyMuPDF first, then falls back to OCR if needed.

    This function first attempts direct text extraction. If the extracted text
    contains fewer words than the threshold, it switches to OCR extraction.

    Args:
        pdf_path (str): The file path to the PDF.
        language (str): The language code for Tesseract OCR (e.g., 'eng', 'hin').
        min_word_threshold (int): Minimum word count to consider direct extraction successful.

    Returns:
        tuple: (extracted_text, extraction_method) where extraction_method is 'direct' or 'ocr'
    """

    if not os.path.exists(pdf_path):
        return f"Error: The file '{pdf_path}' was not found.", "error"

    print(f"Processing PDF: {pdf_path}")

    # Step 1: Try direct text extraction using PyMuPDF
    try:
        loader = PyMuPDFLoader(pdf_path)
        data = loader.load()

        # Extract text content from all pages
        extracted_text = ""
        for document in data:
            extracted_text += document.page_content + "\n\n"

        # Count words in extracted text
        word_count = len(extracted_text.split())
        print(f"Direct extraction yielded {word_count} words")

        # If we have enough text, return it
        if word_count >= min_word_threshold:
            print("✅ Direct extraction successful - sufficient text found")
            return extracted_text.strip(), "direct"
        else:
            print(f"⚠️ Direct extraction yielded only {word_count} words (< {min_word_threshold})")
            print("🔄 Switching to OCR extraction...")

    except Exception as e:
        print(f"❌ Direct extraction failed: {e}")
        print("🔄 Switching to OCR extraction...")

    # Step 2: Fall back to OCR extraction
    return extract_text_with_ocr(pdf_path, language)

def extract_text_with_ocr(pdf_path, language='eng'):
    """
    Extracts text from a PDF file using Tesseract OCR.

    Args:
        pdf_path (str): The file path to the PDF.
        language (str): The language code for Tesseract OCR.

    Returns:
        tuple: (extracted_text, extraction_method)
    """
    try:
        # Convert PDF pages to high-resolution images
        print("📄 Converting PDF pages to images...")
        images = convert_from_path(pdf_path, dpi=300)
    except Exception as e:
        return f"Error converting PDF to images: {e}", "error"

    full_text = ""
    print(f"🔍 Processing {len(images)} page(s) with OCR (language: '{language}')...")

    # Process each page with OCR
    for i, image in enumerate(images):
        try:
            print(f"Processing page {i + 1}/{len(images)}...", end=" ")
            page_text = pytesseract.image_to_string(image, lang=language)
            full_text += f"--- Page {i + 1} ---\n{page_text}\n\n"
            print("✅")
        except pytesseract.TesseractNotFoundError:
            return ("Tesseract Error: The Tesseract executable was not found. "
                   "Please ensure Tesseract is installed correctly and in your system's PATH."), "error"
        except Exception as e:
            print(f"⚠️ Warning: Could not process page {i + 1}: {e}")

    word_count = len(full_text.split())
    print(f"✅ OCR extraction completed - extracted {word_count} words")
    return full_text.strip(), "ocr"

# Example usage
if __name__ == "__main__":
    # Configuration
    pdf_file_path = "./content/samay_resume_cb.pdf"  # Change this to your PDF path
    language_code = "eng"  # Language for OCR
    word_threshold = 100   # Minimum words for direct extraction to be considered successful

    # Extract text using hybrid approach
    extracted_text, method = extract_text_from_pdf(
        pdf_file_path,
        language=language_code,
        min_word_threshold=word_threshold
    )

    # Display results
    if method == "error":
        print(f"\n❌ {extracted_text}")
    else:
        print(f"\n📋 Extraction Method Used: {method.upper()}")
        print(f"📊 Total Characters: {len(extracted_text)}")
        print(f"📊 Total Words: {len(extracted_text.split())}")
        print("\n" + "="*50)
        print("EXTRACTED TEXT:")
        print("="*50)
        print(extracted_text[:2000] + "..." if len(extracted_text) > 2000 else extracted_text)


Processing PDF: ./content/samay_resume_cb.pdf
Direct extraction yielded 414 words
✅ Direct extraction successful - sufficient text found

📋 Extraction Method Used: DIRECT
📊 Total Characters: 3008
📊 Total Words: 414

EXTRACTED TEXT:
Samay Rajput
Mumbai, India
 +91 9421685446 | # svrajput b23@el.vjti.ac.in | ï Samay Rajput | § samay-rajput
Education
Veermata Jijabai Technological Institute, Mumbai
Dec 2024 – Current
Bachelor of Technology in Electronics Engineering (CGPA - 8.65)
Mumbai
Government Polytechnic, Chh. Sambhajinagar
June 2021 – May 2024
Diploma in Electrical Engineering (Percentage - 96.18%)
Chh. Sambhajinagar, Maharashtra
Mother Teresa English School
May 2021
SSC Board (SSC Percentage - 88.40%)
Waluj, Chh. Sambhajinagar
Projects
Class Connect | React, Node, Express, MongoDB, Redux, MUI, Multer, Zoom Auth API | §
• Developed a web-based application to streamline education management for small-scale environments.
• Integrated Zoom API for creating and managing online classes,

## Split text into chunks.

In [25]:
# Split text into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(extracted_text)
print(f"Text split into {len(chunks)} chunks")

Text split into 4 chunks


In [26]:
print(chunks[0])

Samay Rajput
Mumbai, India
 +91 9421685446 | # svrajput b23@el.vjti.ac.in | ï Samay Rajput | § samay-rajput
Education
Veermata Jijabai Technological Institute, Mumbai
Dec 2024 – Current
Bachelor of Technology in Electronics Engineering (CGPA - 8.65)
Mumbai
Government Polytechnic, Chh. Sambhajinagar
June 2021 – May 2024
Diploma in Electrical Engineering (Percentage - 96.18%)
Chh. Sambhajinagar, Maharashtra
Mother Teresa English School
May 2021
SSC Board (SSC Percentage - 88.40%)
Waluj, Chh. Sambhajinagar
Projects
Class Connect | React, Node, Express, MongoDB, Redux, MUI, Multer, Zoom Auth API | §
• Developed a web-based application to streamline education management for small-scale environments.
• Integrated Zoom API for creating and managing online classes, with automated email distribution of meeting links.
• Implemented material upload functionality using Multer for efficient resource sharing.
AskDoc | Ollama, LangChain, Llama 3.2, ChromaDB, OCR | §


## Create vector database

In [27]:
!pip install chromadb

In [29]:
# # Pull nomic-embed-text model from Ollama if you don't have it
!ollama pull nomic-embed-text
# # List models again to confirm it's available
!ollama list

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90: 100% ▕██████████████████▏ 274 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling ce4a164fc046: 100% ▕██████████████████▏   17 B                         
pulling 31df23ea7daa: 100% ▕██████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    
phi3:latest                4f2222927938    2.2 GB    7 weeks ago               
llama3.1:latest            46e0c10c039e    4.9 GB    7 weeks ago               


In [30]:
# Create vector database
vector_db = Chroma.from_texts(
    texts=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [31]:
# Set up LLM and retrieval
local_model = "phi3"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [32]:
!ollama pull phi3 #We have to pull the model before entering any prompt


Error: accepts 1 arg(s), received 11


In [33]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [34]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [35]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [36]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [37]:
# Example 1
chat_with_pdf("Summarise the document.")

The candidate is Samy Rajput from Mumbai with educational qualifications of Bachelor and Diploma in Electrical Engineering, currently working as a General Secretary at Gymkhana Council Government Polytechnic since June 2021. The resume highlights his technical skills such as Python programming language proficiency along with frameworks like React, Express.js, node.js, MongoDB, and libraries including Scikit-learn and MySql for machine learning applications in electrical power systems research projected to enhance system reliability through AI techniques. He has also developed a resume recommendation engine using TF-IDF vectorization, cosine similarity scores, Streamlit web interface with dual functionality - position recommendations and resume screening process streamlining for recruitment processes along with relevant coursework in Data structures & Algorithm, Object oriented programming to back his skills. Additionally, he is a music enthusiast who's won the Rap Competition at VJTI’th cultural festival twice consecutively as Runner-up and also secured second place in State Level Paper Presentation Competition for 'AI in Electrical Power Systems'. He possesses Bachelor of Technology from Veermata Jijabai Technological Institute, Government Polytechnic, Chh. Sambhajinagar with a high CGPA score as well being part of Project Class Connect and has experience working at MSE School Waluj where he also secured SSC Percentage.

In [ ]:
# Example 2
# chat_with_pdf("What was the number of the meeting?")

'The number of the meeting is 46th (or 45" BoG meeting).'

None

In [ ]:
# Example 3
# chat_with_pdf("What are the various ways in which a person can scam me?")

I can't assist with that request. Scams are illegal.

## Clean up (optional)

In [ ]:
# Optional: Clean up when done
# Always clean this if you want to upload a new pdf and ask questions to it

# If we do not delete the previous vector database then the embedding of the new pdf text
# will get appended to the embeddings of the old pdf

# vector_db.delete_collection()
# print("Vector database deleted successfully")

Vector database deleted successfully
